In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from IPython.display import display
from nltk import TweetTokenizer
import pickle
from gensim.models.keyedvectors import KeyedVectors
import time
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation
from keras import utils
from keras.layers import Embedding, Flatten, Concatenate, concatenate, Input
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, Lambda
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras import callbacks
from keras.preprocessing import sequence
from keras import metrics
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from gensim.models import Word2Vec
from lda import LDA

Using TensorFlow backend.


In [2]:
%%time

#labels
dic_aggression_level = {
    'NAG' : 1,
    'CAG' : 2,
    'OAG' : 3
}

dic_reverse_aggression_level = {}
for i in dic_aggression_level:
    dic_reverse_aggression_level[dic_aggression_level[i]] = i
    
print(dic_aggression_level, '\n', dic_reverse_aggression_level)

#train data
train_pd = shuffle(pd.concat((pd.read_csv("train.csv")[['Data', 'Label']], pd.read_csv("valid.csv")[['Data', 'Label']])), random_state=20)
train_pd['Label_num'] = train_pd.Label.map(dic_aggression_level)

#test data
test_fb_pd = shuffle(pd.read_csv("test_fb.csv")[['Data', 'Label']], random_state=20)
test_fb_pd['Label_num'] = test_fb_pd.Label.map(dic_aggression_level)
test_tw_pd = shuffle(pd.read_csv("test_tw.csv")[['Data', 'Label']], random_state=20)
test_tw_pd['Label_num'] = test_tw_pd.Label.map(dic_aggression_level)

#word2vec
word2vec_model = KeyedVectors.load("glove.twitter.27B/glove.twitter.27B.200d.txt.vector.bin")
vocabulary = set(word2vec_model.wv.vocab.keys())
# tokenizer = TweetTokenizer()
# def create_tokens(x):
#     return tokenizer.tokenize(x.lower())
# word2vec_model = Word2Vec(list(map(create_tokens, train_pd['Data'])), size=200, window=5, min_count=1, workers=50)
# vocabulary = set(word2vec_model.wv.vocab.keys())

print("TRAIN DATA")
print(train_pd.Label.value_counts())
display(train_pd.head(10))

print("\n\n\nTEST DATA")
print(test_fb_pd.Label.value_counts())
display(test_fb_pd.head(10))

{'OAG': 3, 'NAG': 1, 'CAG': 2} 
 {1: 'NAG', 2: 'CAG', 3: 'OAG'}
TRAIN DATA
NAG    6285
CAG    5297
OAG    3419
Name: Label, dtype: int64


/opt/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


,Data,Label,Label_num
2930,Focus on making cash available then only peo...,CAG,2
5103,She's so ignorant Megha Mukherji,OAG,3
5090,"Sonia I am holding Rel cap 430, please suggest...",NAG,1
9074,why dont u make ur room sound proof..simple,OAG,3
6769,Showing everything and saying bold...,CAG,2
140,Then what happens in pantry coach dedicated fo...,OAG,3
1756,We should respect every religion. May be he wa...,OAG,3
7272,Car is good.. bt i must say.. i only heard 'aa...,NAG,1
10305,friends we have to understand the ground reali...,CAG,2
45,Bad...........,CAG,2





TEST DATA
NAG    630
OAG    144
CAG    142
Name: Label, dtype: int64


,Data,Label,Label_num
156,What will be done to the money we have ?,NAG,1
211,Unchange the rapo rate could lead the stagnate...,NAG,1
13,PK Movie Bhagawan Shiv Ko Aapman Kiya.. I Ha...,NAG,1
798,Thousands of people have died due to bandhs an...,CAG,2
640,Worst F.M ever.what about 5 lakhs tax limit? ...,NAG,1
568,Pak army rape modi daughter and go back. Now m...,NAG,1
321,I am clean without cash as transactions are th...,NAG,1
119,I have 5000 shares of Pnb @75.90 please tell m...,NAG,1
820,these bhagwa terrorists can't digest their mea...,OAG,3
721,Worst Decision and very worst implementation. ...,OAG,3


CPU times: user 3.26 s, sys: 660 ms, total: 3.92 s
Wall time: 3.89 s


In [3]:
%%time
tweeter = TweetTokenizer()

def get_sentence_vector(sentence, num_words=-1):
    global random_listy
    listy = []
    for token in tweeter.tokenize(sentence.lower()):
        if token in vocabulary:
            listy.append(word2vec_model[token])  
    
#     random_listy.append(len(listy))
#     if len(listy) == 0:
#         print(sentence)
    if num_words != -1:
        listy = listy[:num_words]
    if len(listy) < num_words:
        listy = listy + [[0]*word2vec_model['king'].shape[0]]*(num_words-len(listy))
    
    return np.array(listy).reshape((1, -1))[0]
         
# number_of_words_per_sentence = 81
number_of_words_per_sentence = (sum([len(tweeter.tokenize(sentence)) for sentence in train_pd['Data']]) // train_pd['Data'].shape[0]) * 3
X_train_ = np.array([get_sentence_vector(sentence, number_of_words_per_sentence) for sentence in train_pd['Data']])
y_train_ = utils.to_categorical(train_pd['Label_num']-1)
X_fb_test = np.array([get_sentence_vector(sentence, number_of_words_per_sentence) for sentence in test_fb_pd['Data']])
X_tw_test = np.array([get_sentence_vector(sentence, number_of_words_per_sentence) for sentence in test_tw_pd['Data']])
y_fb_test = test_fb_pd['Label_num']
y_tw_test = test_tw_pd['Label_num']
num_classes = y_train_.shape[1]
print("There are {} words per sentence.".format(number_of_words_per_sentence))
print("X_train: {}\ny_train: {}".format(X_train_.shape, y_train_.shape))
print("X_fb_test: {}\ny_fb_test: {}".format(X_fb_test.shape, y_fb_test.shape))
print("X_tw_test: {}\ny_tw_test: {}".format(X_tw_test.shape, y_tw_test.shape))

There are 81 words per sentence.
X_train: (15001, 16200)
y_train: (15001, 3)
X_fb_test: (916, 16200)
y_fb_test: (916,)
X_tw_test: (1257, 16200)
y_tw_test: (1257,)
CPU times: user 13.9 s, sys: 4.9 s, total: 18.8 s
Wall time: 18.8 s


# POS TAGGING

In [4]:
%%time
from multiprocessing import Pool

#tokenize
from nltk.tokenize import TweetTokenizer
tweeter = TweetTokenizer()
process_pool = Pool(processes=55)
pos_train = list(process_pool.map(tweeter.tokenize, train_pd['Data']))
pos_fb_test  = list(process_pool.map(tweeter.tokenize, test_fb_pd['Data']))
pos_tw_test  = list(process_pool.map(tweeter.tokenize, test_tw_pd['Data']))
process_pool.close()
print("Finished tokenising")

#remove stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def f(x):
    return tuple(filter(lambda y: y not in stop_words, x))
process_pool = Pool(processes=55)
pos_train = list(process_pool.map(f, pos_train))
pos_fb_test = list(process_pool.map(f , pos_fb_test))
pos_tw_test = list(process_pool.map(f , pos_tw_test))
process_pool.close()
print("Finished removing stop words")

#stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()
def f(x):
    return tuple(map(ps.stem, x))
process_pool = Pool(processes=55)
pos_train = list(process_pool.map(f, pos_train))
pos_fb_test = list(process_pool.map(f , pos_fb_test))
pos_tw_test = list(process_pool.map(f , pos_tw_test))
process_pool.close()
print("Finished stemming")

#POS tagging
from nltk import pos_tag
process_pool = Pool(processes=55)
pos_train = list(process_pool.map(pos_tag, pos_train))
pos_fb_test = list(process_pool.map(pos_tag, pos_fb_test))
pos_tw_test = list(process_pool.map(pos_tag, pos_tw_test))
process_pool.close()
print("Finished POS tagging")

#recombination
# def f(x): #pos concatenation
#     return " ".join(list(map("".join, x)))

def f(x): #pos only sentences
    return " ".join(list(map(lambda y: y[1], x)))

process_pool = Pool(processes=55)
pos_train = list(process_pool.map(f, pos_train))
pos_fb_test = list(process_pool.map(f, pos_fb_test))
pos_tw_test = list(process_pool.map(f, pos_tw_test))
process_pool.close()
print("Finished recombining")


#create tfidf matrix
tfidf = TfidfVectorizer()
pos_train = tfidf.fit_transform(pos_train)
pos_fb_test = tfidf.transform(pos_fb_test)
pos_tw_test = tfidf.transform(pos_tw_test)

# print(pos_train[:10], pos_test[:10])
# print(pos_train.toarray().shape)

X_train_ = np.hstack((X_train_, pos_train.toarray()))
X_fb_test = np.hstack((X_fb_test, pos_fb_test.toarray()))
X_tw_test = np.hstack((X_tw_test, pos_tw_test.toarray()))
print(X_train_.shape, y_train_.shape, X_fb_test.shape, y_fb_test.shape, X_tw_test.shape, y_tw_test.shape)

Finished tokenising
Finished removing stop words
Finished stemming
Finished POS tagging
Finished recombining
(15001, 16233) (15001, 3) (916, 16233) (916,) (1257, 16233) (1257,)
CPU times: user 2.21 s, sys: 16.7 s, total: 18.9 s
Wall time: 20.3 s


# LDA

In [5]:
%%time
topics = 150
lda_vect = LDA(n_topics=topics, n_iter=100)
count_vect = CountVectorizer()

count_train = count_vect.fit_transform(train_pd['Data'])
lda_train = lda_vect.fit_transform(count_train)
X_train_ = np.hstack((X_train_, lda_train))
X_fb_test = np.hstack((X_fb_test, lda_vect.transform(count_vect.transform(test_fb_pd['Data']))))
X_tw_test = np.hstack((X_tw_test, lda_vect.transform(count_vect.transform(test_tw_pd['Data']))))

print(X_train_.shape, y_train_.shape, X_fb_test.shape, y_fb_test.shape, X_tw_test.shape, y_tw_test.shape)

INFO:lda:n_documents: 15001
INFO:lda:vocab_size: 24374
INFO:lda:n_words: 354658
INFO:lda:n_topics: 150
INFO:lda:n_iter: 100
/home/Abhor/.local/lib/python3.5/site-packages/lda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -5043427
INFO:lda:<10> log likelihood: -3490021
INFO:lda:<20> log likelihood: -3324977
INFO:lda:<30> log likelihood: -3253029
INFO:lda:<40> log likelihood: -3211266
INFO:lda:<50> log likelihood: -3188011
INFO:lda:<60> log likelihood: -3170713
INFO:lda:<70> log likelihood: -3161906
INFO:lda:<80> log likelihood: -3149490
INFO:lda:<90> log likelihood: -3140927
INFO:lda:<99> log likelihood: -3136607


(15001, 16383) (15001, 3) (916, 16383) (916,) (1257, 16383) (1257,)
CPU times: user 47.1 s, sys: 4.48 s, total: 51.6 s
Wall time: 51.5 s


In [ ]:
save_folder = "results/NN_word2vec/"

#callbacks
callback_list = []
callback_list.append(callbacks.EarlyStopping(monitor='val_acc', min_delta=0.003, patience=4, verbose=0, mode='auto', baseline=None))


def createCNNModel(layer_node_list):
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    input_crap = Input(shape=(X_train.shape[1], 1))
    def split_first(x):
        return x[:, :-183, :]
    def split_second(x):
        return x[:, -183:, :]
    input_first = Lambda(split_first)(input_crap)
    input_second = Lambda(split_second)(input_crap)
    input_second = Flatten()(input_second)
#     print(input_crap.shape, input_first.shape, input_second.shape)
#     print(type(input_crap), type(input_first), type(input_second))
    convo_layers = [Conv1D(filters=8, #!!!!!!!!!!!!!!!!!!!
                         kernel_size=kernel_size,
                         padding='same',
                         activation='relu',
                         strides=200,
                         input_shape=(X_train.shape[1], 1),
                          )(input_first) for kernel_size in range(200, 1001, 200)]
    max_pool_layers = [MaxPooling1D(pool_size=3, strides=None,
                        padding='valid')(convo_layer) for convo_layer in convo_layers]
    merged_layer = concatenate(max_pool_layers)
#     print(merged_layer.shape)
    flattened_layer = Flatten()(merged_layer)
    input_merged_again = concatenate([flattened_layer, input_second])
#     print(flattened_layer.shape)
    fc1 = Dense(layer_node_list[0], activation='relu', kernel_initializer='he_normal')(input_merged_again)
    dropout1 = Dropout(0.5)(fc1)
    fc2 = Dense(layer_node_list[1], activation='relu', kernel_initializer='he_normal') (dropout1)
    dropout2 = Dropout(0.3)(fc2)
#     fc3 = Dense(layer_node_list[2], activation='relu', kernel_initializer='he_normal') (dropout2)
#     dropout3 = Dropout(0.5)(fc3)
    fc4 = Dense(3, activation='softmax')(dropout2)
    model = Model(inputs=input_crap, outputs=fc4)
    model.summary()
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model
    
    

# create our CNN model
X_train, X_valid, y_train, y_valid = train_test_split(X_train_.reshape(X_train_.shape[0], -1, 1), y_train_, test_size=0.2)
print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)
for nodes1 in range(100, 1501, 100):
    for nodes2 in range(100, 1501, 100):
        print(nodes1, nodes2)
        model = createCNNModel([nodes1, nodes2])
        model.fit(X_train, y_train, validation_data=(X_valid, y_valid), nb_epoch=15, batch_size=128,
                 callbacks=callback_list)
        # Final evaluation of the model
        print("\n\n")
        y_pred_fb = model.predict(X_fb_test.reshape(X_fb_test.shape[0], -1, 1))
        y_pred_tw = model.predict(X_tw_test.reshape(X_tw_test.shape[0], -1, 1))
        y_pred_fb = np.array([y_pred_fb[i].argmax()+1 for i in range(len(y_pred_fb))])
        y_pred_tw = np.array([y_pred_tw[i].argmax()+1 for i in range(len(y_pred_tw))])

        print("FB: {}, {} {}".format(accuracy_score(y_fb_test, y_pred_fb), f1_score(y_fb_test, y_pred_fb, average='weighted'),
                                 confusion_matrix(y_fb_test, y_pred_fb)))
        print("TW: {}, {} {}".format(accuracy_score(y_tw_test, y_pred_tw), f1_score(y_tw_test, y_pred_tw, average='weighted'),
                                 confusion_matrix(y_tw_test, y_pred_tw)))
        del model, y_pred_fb, y_pred_tw

(12000, 16383, 1) (12000, 3) (3001, 16383, 1) (3001, 3)
100 100
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 16383, 1)     0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 16200, 1)     0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 81, 8)        1608        lambda_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 81, 8)        3208        lambda_1[0][0]                   
_____________________________________________

/opt/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:60: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 12000 samples, validate on 3001 samples
Epoch 1/15
12000/12000 [==============================] - 10s 792us/step - loss: 0.2065 - acc: 0.4616 - binary_accuracy: 0.6842 - val_loss: 0.1888 - val_acc: 0.5178 - val_binary_accuracy: 0.7123
Epoch 2/15
12000/12000 [==============================] - 8s 690us/step - loss: 0.1844 - acc: 0.5395 - binary_accuracy: 0.7158 - val_loss: 0.1820 - val_acc: 0.5442 - val_binary_accuracy: 0.7209
Epoch 3/15
12000/12000 [==============================] - 8s 678us/step - loss: 0.1741 - acc: 0.5784 - binary_accuracy: 0.7320 - val_loss: 0.1778 - val_acc: 0.5655 - val_binary_accuracy: 0.7293
Epoch 4/15
12000/12000 [==============================] - 8s 679us/step - loss: 0.1612 - acc: 0.6263 - binary_accuracy: 0.7573 - val_loss: 0.1802 - val_acc: 0.5675 - val_binary_accuracy: 0.7268
Epoch 5/15
12000/12000 [==============================] - 8s 671us/step - loss: 0.1458 - acc: 0.6757 - binary_accuracy: 0.7894 - val_loss: 0.1848 - val_acc: 0.5691 - val_bina

12000/12000 [==============================] - 7s 586us/step - loss: 0.1438 - acc: 0.6813 - binary_accuracy: 0.7904 - val_loss: 0.1894 - val_acc: 0.5618 - val_binary_accuracy: 0.7141
Epoch 6/15
12000/12000 [==============================] - 7s 597us/step - loss: 0.1259 - acc: 0.7316 - binary_accuracy: 0.8247 - val_loss: 0.1964 - val_acc: 0.5555 - val_binary_accuracy: 0.7129
Epoch 7/15
12000/12000 [==============================] - 7s 581us/step - loss: 0.1107 - acc: 0.7672 - binary_accuracy: 0.8469 - val_loss: 0.2124 - val_acc: 0.5408 - val_binary_accuracy: 0.7005
Epoch 8/15
12000/12000 [==============================] - 7s 596us/step - loss: 0.0958 - acc: 0.8084 - binary_accuracy: 0.8717 - val_loss: 0.2220 - val_acc: 0.5352 - val_binary_accuracy: 0.6961



FB: 0.5305676855895196, 0.5744455008332229 [[335 236  59]
 [ 37  77  28]
 [ 20  50  74]]
TW: 0.4693715194908512, 0.46658424317659564 [[289 156  38]
 [156 181  76]
 [ 82 159 120]]
100 300
_____________________________________________

lambda_8 (Lambda)               (None, 183, 1)       0           input_4[0][0]                    
__________________________________________________________________________________________________
flatten_8 (Flatten)             (None, 1080)         0           concatenate_7[0][0]              
__________________________________________________________________________________________________
flatten_7 (Flatten)             (None, 183)          0           lambda_8[0][0]                   
__________________________________________________________________________________________________
concatenate_8 (Concatenate)     (None, 1263)         0           flatten_8[0][0]                  
                                                                 flatten_7[0][0]                  
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 100)          126400      concatenate_8[0][0]              
__________

...

12000/12000 [==============================] - 9s 711us/step - loss: 0.0941 - acc: 0.8059 - binary_accuracy: 0.8719 - val_loss: 0.2159 - val_acc: 0.5435 - val_binary_accuracy: 0.7008



FB: 0.5109170305676856, 0.5537123333550418 [[330 220  80]
 [ 49  62  31]
 [ 21  47  76]]
TW: 0.4677804295942721, 0.4668073519388786 [[281 159  43]
 [154 171  88]
 [ 87 138 136]]
200 100
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 16383, 1)     0                                            
__________________________________________________________________________________________________
lambda_31 (Lambda)              (None, 16200, 1)     0           input_16[0][0]                   
__________________________________________________________________________________________________
conv1d_76 (Conv1D)              (N

Train on 12000 samples, validate on 3001 samples
Epoch 1/15
12000/12000 [==============================] - 10s 867us/step - loss: 0.2029 - acc: 0.4713 - binary_accuracy: 0.6893 - val_loss: 0.1856 - val_acc: 0.5392 - val_binary_accuracy: 0.7151
Epoch 2/15
12000/12000 [==============================] - 8s 687us/step - loss: 0.1800 - acc: 0.5502 - binary_accuracy: 0.7208 - val_loss: 0.1808 - val_acc: 0.5501 - val_binary_accuracy: 0.7218
Epoch 3/15
12000/12000 [==============================] - 8s 688us/step - loss: 0.1671 - acc: 0.5987 - binary_accuracy: 0.7439 - val_loss: 0.1826 - val_acc: 0.5538 - val_binary_accuracy: 0.7224
Epoch 4/15
12000/12000 [==============================] - 8s 688us/step - loss: 0.1526 - acc: 0.6486 - binary_accuracy: 0.7710 - val_loss: 0.1784 - val_acc: 0.5668 - val_binary_accuracy: 0.7260
Epoch 5/15
12000/12000 [==============================] - 8s 692us/step - loss: 0.1333 - acc: 0.7069 - binary_accuracy: 0.8087 - val_loss: 0.1858 - val_acc: 0.5588 - val_bina

12000/12000 [==============================] - 8s 660us/step - loss: 0.1145 - acc: 0.7560 - binary_accuracy: 0.8399 - val_loss: 0.2059 - val_acc: 0.5345 - val_binary_accuracy: 0.6987
Epoch 7/15
12000/12000 [==============================] - 8s 657us/step - loss: 0.0967 - acc: 0.8003 - binary_accuracy: 0.8686 - val_loss: 0.2174 - val_acc: 0.5418 - val_binary_accuracy: 0.6997
Epoch 8/15
12000/12000 [==============================] - 8s 666us/step - loss: 0.0804 - acc: 0.8381 - binary_accuracy: 0.8932 - val_loss: 0.2216 - val_acc: 0.5455 - val_binary_accuracy: 0.7057



FB: 0.5644104803493449, 0.5968348053941722 [[372 167  91]
 [ 47  71  24]
 [ 29  41  74]]
TW: 0.48289578361177404, 0.4759284073585877 [[308 130  45]
 [179 128 106]
 [ 94  96 171]]
200 400
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 16

Train on 12000 samples, validate on 3001 samples
Epoch 1/15
12000/12000 [==============================] - 11s 914us/step - loss: 0.2024 - acc: 0.4779 - binary_accuracy: 0.6889 - val_loss: 0.1880 - val_acc: 0.5245 - val_binary_accuracy: 0.7133
Epoch 2/15
12000/12000 [==============================] - 8s 708us/step - loss: 0.1801 - acc: 0.5549 - binary_accuracy: 0.7229 - val_loss: 0.1809 - val_acc: 0.5541 - val_binary_accuracy: 0.7260
Epoch 3/15
12000/12000 [==============================] - 8s 700us/step - loss: 0.1661 - acc: 0.6072 - binary_accuracy: 0.7464 - val_loss: 0.1791 - val_acc: 0.5695 - val_binary_accuracy: 0.7292
Epoch 4/15
12000/12000 [==============================] - 8s 700us/step - loss: 0.1480 - acc: 0.6630 - binary_accuracy: 0.7850 - val_loss: 0.1860 - val_acc: 0.5538 - val_binary_accuracy: 0.7169
Epoch 5/15
12000/12000 [==============================] - 8s 705us/step - loss: 0.1289 - acc: 0.7178 - binary_accuracy: 0.8179 - val_loss: 0.1936 - val_acc: 0.5521 - val_bina

11904/12000 [============================>.] - ETA: 0s - loss: 0.0970 - acc: 0.8020 - binary_accuracy: 0.8699